## Milestone 1 - Plot the World

### Tasks: Load a world countries geojson
###        Display using folium
###        Add base location as point marker

In [ ]:
"""
**************************************************************************
*
* Author: Cooper Wolf
* Email: ctwolf1014@my.msutexas.edu
* Label: P01
* Title: Missile Geometry
* Course: CMPS 5993
* Semester: Spring 2026
*
* Description:
* This noteboook visualizes the geometry of a missile's range using Folium and GeoPandas.
* It displays a world map with country boundaries and marks the base location from which the missile is launched.
*
* 
*
* Files:
* Missile_Geometry.ipynb        : driver program
* countries.geo.json            : GeoJSON file containing world country boundaries
*
**************************************************************************
"""

In [ ]:
# Installing libraries
%pip install folium geopandas

### Load world geojson

In [2]:
# Importing libraries
import geopandas as gpd
import folium

# Reading the world data
world = gpd.read_file("countries.geojson")

# Creating a map
m = folium.Map(location=[20,0], zoom_start=2)

### Display using folium

In [3]:
# Converting GeoDataFrame to GeoJSON and adding to Folium
folium.GeoJson(world, name="World Countries").add_to(m)

### Adding base location as point marker

In [4]:
# Adding base location as a point marker
base_lat = 38.89767
base_lon = -77.03655

# Adding a marker for the base location
folium.Marker(location=[base_lat, base_lon], popup="Base", icon=folium.Icon(color="red", icon="home")).add_to(m)

In [ ]:
m

## Milestone 2 - Distance & Bearing

### Task: Compute distance from each threat origin to base
###       Identify closest threat
###       Display threat origin as points

#### Helper functions graciously provided by Dr. G

In [5]:
from __future__ import annotations
import math
from typing import Tuple


# ----------------------------------------
# Constants
# ----------------------------------------
EARTH_RADIUS_KM = 6371.0088  # mean Earth radius


# ----------------------------------------
# Distance: Haversine (km)
# ----------------------------------------
def haversine_km(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Compute great-circle distance between two lat/lon points.
    Returns distance in kilometers.
    """
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)

    dphi = math.radians(lat2 - lat1)
    dlmb = math.radians(lon2 - lon1)

    a = (
        math.sin(dphi / 2) ** 2
        + math.cos(phi1) * math.cos(phi2) * math.sin(dlmb / 2) ** 2
    )
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return EARTH_RADIUS_KM * c


# ----------------------------------------
# Destination point (bearing + distance)
# ----------------------------------------
def destination_point(
    lat: float, lon: float, bearing_deg: float, distance_km: float
) -> Tuple[float, float]:
    """
    Compute destination lat/lon given:
      - start lat/lon
      - bearing in degrees
      - distance in km

    Uses great-circle formula.
    """
    bearing = math.radians(bearing_deg)

    lat1 = math.radians(lat)
    lon1 = math.radians(lon)
    d = distance_km / EARTH_RADIUS_KM

    lat2 = math.asin(
        math.sin(lat1) * math.cos(d) + math.cos(lat1) * math.sin(d) * math.cos(bearing)
    )

    lon2 = lon1 + math.atan2(
        math.sin(bearing) * math.sin(d) * math.cos(lat1),
        math.cos(d) - math.sin(lat1) * math.sin(lat2),
    )

    # normalize longitude to [-180, 180]
    lon2 = (math.degrees(lon2) + 540) % 360 - 180
    lat2 = math.degrees(lat2)

    return lat2, lon2

### Computing distance from each threat to base
### Identifing closest threats

In [22]:
## Read in threats, calculate each distance from base, and identify closest threat, and display their original location as points
import json


# Read in threats from JSON file
with open("threats.json", "r") as f:
    threats = json.load(f)
    
# Calculate distance from base for each threat and store in dictionary
for threat in threats:
    threat["dist_from_base"] = haversine_km(base_lat, base_lon, threat["origin_lat"], threat["origin_lon"])
    
# Printing threat and its distance from base
print("Threat distances from base:")
for threat in threats:
    print(f"Threat ID: {threat["id"]}, Distance from base: {threat["dist_from_base"]:.2f} km")

# Identify closest threat    
threats_ordered_by_distance = sorted(threats, key=lambda x: x["dist_from_base"])
print("\nThreats ordered by distance from base:")

for threat in threats_ordered_by_distance:
    print(f"Threat ID: {threat["id"]}, Distance from base: {threat["dist_from_base"]:.2f} km")


Threat distances from base:
Threat ID: T001, Distance from base: 2512.64 km
Threat ID: T002, Distance from base: 4880.88 km
Threat ID: T003, Distance from base: 4703.22 km
Threat ID: T004, Distance from base: 4874.54 km
Threat ID: T005, Distance from base: 2402.38 km
Threat ID: T006, Distance from base: 2879.01 km
Threat ID: T007, Distance from base: 4806.17 km
Threat ID: T008, Distance from base: 3957.28 km
Threat ID: T009, Distance from base: 2328.06 km
Threat ID: T010, Distance from base: 3626.03 km

Threats ordered by distance from base:
Threat ID: T009, Distance from base: 2328.06 km
Threat ID: T005, Distance from base: 2402.38 km
Threat ID: T001, Distance from base: 2512.64 km
Threat ID: T006, Distance from base: 2879.01 km
Threat ID: T010, Distance from base: 3626.03 km
Threat ID: T008, Distance from base: 3957.28 km
Threat ID: T003, Distance from base: 4703.22 km
Threat ID: T007, Distance from base: 4806.17 km
Threat ID: T004, Distance from base: 4874.54 km
Threat ID: T002, Dis

### Displaying threat origins as points

In [ ]:
from folium.features import DivIcon


# Displaying threat origins as points on the map
for threat in threats:
    
    # Assigning icons based on threat type
    if threat["type"] == "alien":
        icon_html = '<div style="font-size: 24px;">👽</div>'
    elif threat["type"] == "kaiju":
        icon_html = '<div style="font-size: 24px;">🦖</div>'
    elif threat["type"] == "airborne":
        icon_html = '<div style="font-size: 24px;">✈️</div>'
    elif threat["type"] == "orbital":
        icon_html = '<div style="font-size: 24px;">🛰️</div>'

# Adding markers for each threat origin with appropriate icons
    folium.Marker(
        location=[threat["origin_lat"], threat["origin_lon"]],
        popup=f'{threat["id"]} - {threat["type"]}',
        icon=DivIcon(html=icon_html)
    ).add_to(m)

In [ ]:
m

In [45]:
m.save("World_with_threats.html")

## Milestone 3 - Trajectories

### Tasks: For each threat: compute distination point after a fixed time interval
###                         generate intermediate points
###                         construct a LineString trajectory
###        Plot trajectories on the map


#### For each threat:
#### Computing distribution point after fixed time
#### Generating intermediate points
#### Constructing LineString trajectory

In [26]:
# Function  to generate intermediate points
def generate_points(lat1: float, lon1: float, lat2: float, lon2: float, num_points: int) -> list[Tuple[float, float]]:
    """
    Generate intermediate points between two lat/lon coordinates.
    Returns a list of (lat, lon) tuples.
    """
    
    # Create points list
    points = []
    
    # Generate intermediate points using linear interpolation
    for i in range(1, num_points + 1):
        fraction = i / (num_points + 1)
        lat = lat1 + (lat2 - lat1) * fraction
        lon = lon1 + (lon2 - lon1) * fraction
        points.append((lat, lon))
    return points

In [43]:
# Dictionary to store threat id and its destination point
threat_points = {}

# Looping through each threat
for threat in threats:
    threat["dest_lat"], threat["dest_lon"] = destination_point(threat["origin_lat"], threat["origin_lon"], threat["bearing_deg"], threat["dist_from_base"])
    
    # Printing destination points
    print(f"Threat ID: {threat["id"]}, Destination Lat: {threat["dest_lat"]:.4f}, Destination Lon: {threat["dest_lon"]:.4f}")
    
    # Generate intermediate
    threat_points[threat["id"]] = generate_points(threat["origin_lat"], threat["origin_lon"], threat["dest_lat"], threat["dest_lon"], num_points=5)
    

Threat ID: T001, Destination Lat: 36.8800, Destination Lon: -92.5684
Threat ID: T002, Destination Lat: 50.5944, Destination Lon: -93.8280
Threat ID: T003, Destination Lat: 40.5373, Destination Lon: -78.8787
Threat ID: T004, Destination Lat: 34.7698, Destination Lon: -74.2756
Threat ID: T005, Destination Lat: 22.2223, Destination Lon: -92.3439
Threat ID: T006, Destination Lat: 27.0437, Destination Lon: -98.0965
Threat ID: T007, Destination Lat: 23.9547, Destination Lon: -80.3620
Threat ID: T008, Destination Lat: 22.5100, Destination Lon: -91.7222
Threat ID: T009, Destination Lat: 37.9031, Destination Lon: -91.5162
Threat ID: T010, Destination Lat: 28.1274, Destination Lon: -89.5994


In [45]:
# Creating LineString / trajectory for each threat
for threat in threats:
    
    # Default color
    color = "red"
    
    # Assign color based on type
    if threat["type"] == "alien":
        color = "green"
    elif threat["type"] == "kaiju":
        color = "blue"
    elif threat["type"] == "airborne":
        color = "orange"
    elif threat["type"] == "orbital":
        color = "purple"
    
    # Get intermediate points for this threat
    points = threat_points[threat["id"]]  # List of (lat, lon)
    
    # Plot each intermediate point
    for point in points:
        folium.CircleMarker(location=point, radius=3, color="black", fill=True).add_to(m)
    
    # Draw the trajectory line using the same points
    folium.PolyLine(points, color=color, weight=2).add_to(m)


In [46]:
m.save("World_with_threats.html")